In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import sys
import pickle
from itertools import product

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style='whitegrid')
sns.set_theme(style='ticks')

In [ ]:
repo_dir = Path('../..')

In [ ]:
if str(repo_dir) not in sys.path:
    sys.path.append(str(repo_dir))
    
from analysis.curve_fitting.src.fitting_functions import LOSS_FUNCTIONS
from analysis.curve_fitting.src.utils import apply_filters, load_yaml, convert_loss_parameters, convert_loss_parameters_batch

from visualization.src.utils import COLOR_PALETTES, set_ticks, COLORS, save_figs
from visualization.src.visualize import plot_reg, plot_reg_bivariate, plot_confidence_intervals



In [ ]:
args = {
    'results_csv': repo_dir / 'results' / 'benchmark_scores.csv',
}

In [ ]:
results_csv = args['results_csv']

df_results = pd.read_csv(results_csv)


## Load Experiment Configuration

In [ ]:
analysis_dir = repo_dir / 'analysis'
config_dir = analysis_dir / 'curve_fitting/configs/parameter'
results_dir = analysis_dir / 'curve_fitting/outputs/fitting_results'

In [ ]:
experiment_name = "parameter_pretrained_scaling"
config = load_yaml(config_dir / f'{experiment_name}.yaml')


In [ ]:
L_fit = config['fitting_parameters']['loss_function']
L_viz = config['visualization']['loss_function']
x_scaler = float(config['fitting_parameters']['X_scaler'])



## Apply Data Filters

In [ ]:
df = apply_filters(df_results, config.get('data_filters', {}))



# df = df_results[df_results['is_pretrained'] == True]
# df = df[df.model_id.apply(lambda x: 'fcmae' not in x)]
# df = df[df.arch_family != 'ConViT']
# df = df[df.arch_family != 'AlexNet']
# df = df[df.arch_family != 'ViT-B-OpenClip']
# df = df.groupby(['model_id', 'n_params', 'arch', 'arch_family', 'dataset']).agg({'score': 'mean'}).reset_index()
df

In [ ]:
df.arch.nunique()
df.arch_family.unique()
df[df.model_id.apply(lambda x: '_nc-' not in x)].model_id.nunique()

## Load Fitting Results

In [ ]:
with open(results_dir / f'{experiment_name}' / 'results.pkl', 'rb') as f:
    results = pickle.load(f)


optimized_params = convert_loss_parameters(results['optimized_parameters'], L_fit, L_viz)

# Convert bootstrapped parameters
opt_params_boot = results['optimized_parameters_bootstrapped']
opt_params_boot = convert_loss_parameters_batch(
    params=opt_params_boot,
    src_loss=L_fit,
    dst_loss=L_viz
)

## Visualize

#### Plotting settings

In [ ]:
x_extend = 1.1
X_str = r'$$\tilde{N}$$'
linewidth = 3.0
alpha_scatter = 1.0
alpha_ci = 0.2
alpha_fit = 1.0
fig_multiplier = 0.7
figsize = (10, 8)
figsize = (fig_multiplier * figsize[0], fig_multiplier * figsize[1])

color_palaette = COLOR_PALETTES['models']
color = "#023e8a"

#### Plot

In [ ]:
sns.set_theme(style='whitegrid')
sns.set_theme(style='ticks')


fig, axes = plt.subplots(1, 1, figsize=figsize, dpi=300)

ax = axes

### Neuro
df_plot = df
L = LOSS_FUNCTIONS[L_viz]
x_scaler = x_scaler
X = df_plot.n_params.values / x_scaler



df_plot.rename(columns={'arch_family': 'Architecture'}, inplace=True)
sns.scatterplot(data=df_plot, x='n_params', y='score', ax=ax, hue='Architecture', style='Architecture', s=120, palette=color_palaette, alpha=alpha_scatter)
plot_reg(X, optimized_params, L, ax, color=color, x_extend=x_extend, linestyle='-', X_str=X_str, x_scaler=x_scaler, show_x_scaler=False, linewidth=linewidth, legend=True, alpha=alpha_fit)
plot_confidence_intervals(X, opt_params_boot, L, ax, color=color, x_extend=x_extend, x_scaler=x_scaler, alpha=alpha_ci, percentile=95.0, invert_y=True)

### Formatting
ax.set_xscale('log')
ax.set_xlabel('Model Parameters (N)', fontsize=16, fontweight='bold')
ax.set_ylabel('Alignment Score (S)', fontsize=16, fontweight='bold')
# ax.set_title(arch_family, fontsize=20, fontweight='bold')
ax = set_ticks(ax, xticks_mode='log', yticks_mode=None, yticks=[0.2, 0.3, 0.4, 0.5])
ax.set_ylim(0.27, 0.5)

ax.spines[['right', 'top']].set_visible(False)
    
plt.tight_layout()


figures_dir = '../figures'
fig_name = 'fig9_params_pretrained'
formats = ['pdf', 'png', 'svg']
save_figs(figures_dir, fig_name, formats=formats)


In [ ]:
df.sort_values('score', ascending=False)

In [ ]:
df[df.model_id.apply(lambda x: '_nc-' in x)].model_id.unique()

In [ ]:
d = df_results[df_results['is_pretrained'] == True]
d = d[d.region == 'Behavioral']
d = d.sort_values('score', ascending=False)
d[d.columns[:15]]

In [ ]:
# d = df_results[df_results['is_pretrained'] == True]
d = df_results
# d = d[d.region != 'Behavioral']
d = d.groupby(['model_id', 'n_params', 'arch', 'arch_family', 'dataset']).agg({'score': 'mean'}).reset_index()
d = d.sort_values('score', ascending=False)
d[d.columns[:15]]

In [ ]:
list(d.model_id)[:10]

In [ ]:
# d = df_results[df_results['is_pretrained'] == True]
d = df_results
d = d[d.region != 'Behavioral']
d = d.groupby(['model_id', 'n_params', 'arch', 'arch_family', 'dataset']).agg({'score': 'mean'}).reset_index()
d = d.sort_values('score', ascending=False)
d[d.columns[:15]]

In [ ]:
list(d.model_id)[:10]